In [5]:
import numpy as np
import numpy
import scipy
import qiskit
from qiskit import transpile
import scipy.linalg as sl
import numpy.linalg as nl

from scipy.linalg import cossin
from qiskit.quantum_info import Operator, Statevector
from qiskit import QuantumCircuit
from qiskit.circuit.library import UGate,UCPauliRotGate

import sys
sys.path.append('./src')

from lchs import *
from lcu import *

import utils_sp
import importlib
importlib.reload(utils_sp)
from utils_sp import *


np.set_printoptions(precision=8, suppress=True)

In [144]:
def second_decomp(block_u1:numpy.ndarray, block_u2:numpy.ndarray, enable_debug:bool=True) -> (numpy.ndarray, numpy.ndarray, numpy.ndarray):
    ## Accoding to Eq. (16) in 10.1109/TCAD.2005.855930
    ## Cosine-Sine decompsotion gives 
    ## U = [ A_1     ] [C    -S ] [A_2     ]
    ##     [     B_1 ] [S     C ] [    B_2 ]
    ## where A_1, B_1, A_2, B_2 are unitary matrices in equal shapes
    ## Then, this function construct the 2nd decompsotion for each left and right block diagonal matrices
    ## [ U_1    ] = [V   ] [D          ] [W   ]
    ## [     U_2]   [   V] [   D^dagger] [   W]
    ## the constructon is like the following  => U_1 = VDW, U_2 = VD^dagger W
    ## We cancel out W terms and get U_1 U_2^\dagger = V D^2 V^\dagger
    ## Then we diagonalize U_1 U_2^\dagger to obtain V (eigenvector matrix) and D^2 (eigenvalue matrix)
    ## Then D = sqrt(D^2), W = D V^\dagger U_2 -> this is the equation in paper, but W = D^{-1} V^\dagger U_1 is correct
    ## Note that [D          ] is a R_z multiplexer
    ##           [   D^dagger]

    if block_u1.shape[0] != block_u1.shape[1] or block_u2.shape[0] != block_u2.shape[1]:
        raise ValueError('Input matrices must be square, but', block_u1.shape, block_u2.shape, 'were given')
    if block_u1.shape[0] != block_u2.shape[0]:
        raise ValueError('Input matrices must have the same size, but', block_u1.shape[0], block_u2.shape[0], 'were given')
    
    bu_evals, bu_v = scipy.linalg.eig(block_u1.dot( block_u2.T.conj() ) )
    bu_d_inv = np.diag( 1/np.sqrt(bu_evals) )
    bu_w = bu_d_inv @ bu_v.T.conj() @ block_u1

    if enable_debug:
        bu_d = np.diag( np.sqrt(bu_evals) )
        zeroes = np.zeros_like(block_u1)
        prod_mat = np.array([[bu_v, zeroes], [zeroes, bu_v]]) @ np.array([[bu_d, zeroes], [zeroes, bu_d.conj().T]]) @ np.array([[bu_w, zeroes],[zeroes, bu_w]])
        ans = np.array([[block_u1, zeroes], [zeroes, block_u2]])
        print("2nd decomp error", nl.norm(prod_mat - ans))  

    return bu_v, np.sqrt(bu_evals), bu_w



def unitary_synthesis(unitary:numpy.ndarray, circuit:qiskit.QuantumCircuit, bottom_control:list[int], top_target:int, debug=False):
    ## Use uniformly controlled rotations to synthesize a unitary matrix
    ## Based on Quantum Shannon Decomposition (QSD) in [1]
    ##
    ## QSD decomposition gives U = U CS V, 
    ## 2nd decomposition gives U = U_V U_D U_W   and   V = V_V V_D V_W
    ## So the Circuit order: |\phi> [v1 v2] [C S] [u1 u2]
    ##                    -> |\phi> (V_W V_D V_V) CS (U_W U_D U_V)
    ## V_D and U_D are diagonal matrices, so we can use multiplexer R_z gates
    ## CS is multiplexer R_y gates
    ## Each V_W, V_V, U_W, U_V are unitaries, so do the decompsoition recursively
    ##
    ## [1] 10.1109/TCAD.2005.855930
    ## NOTE: For Multiplexer
    ##                      0 ----|R_y|--
    ##                    n-1 -/---ctrl-
    ## bottom_control is [1,2,3,...,n-1], top_target is 0

    (u1,u2), thetas_cs, (v1h, v2h) = scipy.linalg.cossin(unitary, p=unitary.shape[0]//2, q=unitary.shape[1]//2, separate=True)
    # _, cs_mat, _ = scipy.linalg.cossin(unitary, p=unitary.shape[0]//2, q=unitary.shape[1]//2)
    u_v, u_dd, u_w = second_decomp(u1, u2, enable_debug=debug)
    # v_v, v_dd, v_w = second_decomp(v1h.T.conj(), v2h.T.conj(), enable_debug=debug)
    v_v, v_dd, v_w = second_decomp(v1h, v2h, enable_debug=debug)
    thetas_cs = list(thetas_cs*2) ##  WARNING: based on Qiskit implementation, the angles are multiplied by 2 for Multiplexer

    u_zangle = list(np.angle(u_dd)* (-2)) ## R_z(lambda) = exp(-i lambda Z/2)
    v_zangle = list(np.angle(v_dd)* (-2)) ## R_z(lambda) = exp(-i lambda Z/2)

    if debug:
        u1err = nl.norm(u_v @ np.diag(u_dd) @ u_w - u1)
        u2err = nl.norm(u_v @ np.diag(u_dd).conj() @ u_w - u2)
        if u1err > 1e-12:
            raise ValueError('Invalid 2nd decomposition for U, the error is', u1err)
        if u2err > 1e-12:
            raise ValueError('Invalid 2nd decomposition for U, the error is', u2err)
    
    # print("\n", bottom_control, top_target)
    ## Recursively synthesize the unitaries
    if len(bottom_control) == 0:
        circuit.unitary(unitary, top_target)
        return
    ## v
    unitary_synthesis(v_w, circuit, bottom_control[1:], bottom_control[0])
    multiplexer_pauli(circuit, v_zangle, bottom_control, top_target, axis='Z') ## V_D
    unitary_synthesis(v_v, circuit, bottom_control[1:], bottom_control[0])
    # CS
    circuit.barrier()
    multiplexer_pauli(circuit, thetas_cs, bottom_control, top_target, axis='Y') ## V_D
    circuit.barrier()
    ## u
    unitary_synthesis(u_w, circuit, bottom_control[1:], bottom_control[0])
    multiplexer_pauli(circuit, u_zangle, bottom_control, top_target, axis='Z') ## V_D
    unitary_synthesis(u_v, circuit, bottom_control[1:], bottom_control[0])

    # ## v
    # circuit.unitary(scipy.linalg.block_diag(v_w, v_w), bottom_control+[top_target])
    # circuit.unitary(scipy.linalg.block_diag(np.diag(v_dd),np.diag(v_dd).conj()), bottom_control+[top_target])
    # circuit.unitary(scipy.linalg.block_diag(v_v, v_v), bottom_control+[top_target])
    # ## CS
    # circuit.unitary(cs_mat, bottom_control+[top_target])
    # ## u    
    # circuit.unitary(scipy.linalg.block_diag(u_w, u_w), bottom_control+[top_target])
    # circuit.unitary(scipy.linalg.block_diag(np.diag(u_dd),np.diag(u_dd).T), bottom_control+[top_target])
    # circuit.unitary(scipy.linalg.block_diag(u_v, u_v), bottom_control+[top_target])

In [152]:
n = 5

##
rng = np.random.default_rng(9322825479999827034282302141)
psi = rng.random(2**n)
# psi = np.array([0,0.5,0.5,0])
psi = psi / np.linalg.norm(psi, ord=2)#
state0 = numpy.zeros(2**n,dtype=int)
state0[0] = 1 #
##
U = np.eye(2**n, dtype=complex)
U[:, 0] = psi
U,r = numpy.linalg.qr(U, mode='complete')
if numpy.linalg.norm(U[:, 0] - psi[0]) > 1e-8:
    U = -U

print("Ideal error", numpy.linalg.norm(U.dot(state0) - psi))
print("State", psi)

##
qiscirc = QuantumCircuit(n)
qiscirc.initialize(psi)

qiscirc_trans = transpile(qiscirc, basis_gates=['rz','ry','rx', 'cx'], optimization_level=0)
print("Qiskit circuit", dict(qiscirc_trans.count_ops()) )
# qiscirc_trans.draw()

Ideal error 2.1300398239941415e-16
State [0.20432423 0.13516195 0.09312569 0.1702143  0.07317011 0.06456402
 0.05776223 0.07473044 0.22633805 0.12989821 0.05974905 0.20642486
 0.01464517 0.21527556 0.26529004 0.09393781 0.26699801 0.11785323
 0.11869949 0.27421717 0.24747546 0.2250407  0.21252766 0.13163754
 0.28612975 0.04667651 0.02858606 0.20383938 0.19148548 0.06969939
 0.28430822 0.21501397]
Qiskit circuit {'rz': 93, 'rx': 62, 'cx': 26, 'reset': 5}


In [153]:
my_circuit = QuantumCircuit(n)
unitary_synthesis(U, my_circuit, list(range(n))[1:], 0)
my_circuit = my_circuit.reverse_bits()
my_circ_mat = Operator(my_circuit).data

my_circuit_trans = transpile(my_circuit, basis_gates=['rz','ry','rx', 'cx'], optimization_level=1)

print("My Implementation", dict(my_circuit.count_ops()) )
print("Optimized Implemntation", dict(my_circuit_trans.count_ops()) )
print("My error", numpy.linalg.norm(my_circ_mat - U) )

# my_circuit.draw()

My Implementation {'cx': 720, 'rz': 479, 'unitary': 256, 'ry': 240, 'barrier': 170}
Optimized Implemntation {'rz': 991, 'cx': 720, 'ry': 496, 'barrier': 170}
My error 1.0383025598085432e-13


In [143]:
u_sp, cs_sp, v_sp = scipy.linalg.cossin(U, p=U.shape[0]//2, q=U.shape[1]//2)
u_sp.real, cs_sp.real, v_sp.real

(array([[ 0.        , -0.        , -0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [-0.57160229, -0.74857125,  0.33602367, -0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [-0.39383019, -0.10897397, -0.91270064, -0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [-0.71983929,  0.65403804,  0.23252019, -0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        , -0.71953454,
          0.28751996, -0.33063201,  0.53878084],
        [ 0.        ,  0.        ,  0.        ,  0.        , -0.13039714,
         -0.26377747,  0.82909748,  0.47541076],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.22342404,
         -0.75366147, -0.44852367,  0.42532647],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.64447523,
          0.52891219, -0.04589508,  0.55027013]]),
 array([[ 1.  

In [122]:
my_circ_mat.real

array([[ 0.65381679, -0.31362995, -0.62085702,  0.29781944],
       [ 0.43250453,  0.90163176,  0.        ,  0.        ],
       [ 0.62085702, -0.29781944,  0.65381679, -0.31362995],
       [-0.        ,  0.        ,  0.43250453,  0.90163176]])

In [123]:
U.real

array([[ 0.65381679, -0.31362995, -0.25393423,  0.64006036],
       [ 0.43250453,  0.90163176, -0.        , -0.        ],
       [ 0.29799275, -0.1429444 ,  0.94380465,  0.        ],
       [ 0.54466848, -0.2612725 , -0.2115424 , -0.76832463]])

In [100]:
scipy.linalg.block_diag(np.eye(2), np.eye(2))

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])